In [2]:
import numpy as np
import pandas as pd

import plotly.express as px
pd.options.plotting.backend = "plotly"
from scipy.stats import binom,beta

In [69]:
def posterior(A, initial, O):
    # now compute P(X(t) | o_1, ..., o_t)
    forward = np.empty((T, 2))
    normalization =  np.empty(T)
    forward[0] = initial

    for i in range(1, T):
        forward[i] = forward[i - 1] @ A @ np.diag(B[:, int(O[i]) - 2])
        normalization[i] = forward[i].sum()
        forward[i] /= normalization[i]

    # now compute backward P(o_{t + 1}, .. o_T | X_t)
    backward = np.empty((T, 2))
    # in any state of X(T) we have prob 1 of seeing o_T
    # by def
    backward[T - 1] = np.ones(2)

    for i in reversed(range((T - 1))):
        # transpose or left mult because
        # we are going backward in time
        backward[i] = A @ np.diag(B[:, int(O[i + 1]) - 2]) @ backward[i + 1]
        backward[i] /= normalization[i + 1]
    # these will not be normalized as prob dist
    # since we are using normalization to make
    # entire conditional (prod) work
    return forward * backward

In [71]:
# markov transition matrix
A = np.array([[.1, .9],
              [.3, .7]])
# emission probabilities
B = np.array([[.5, .5],
              [.5, .5]])
# one sample up to time T
T = 10
# sample hidden state
X = np.empty(T)
X[0] = 0
for i in range(1, T):
    X[i] = np.random.binomial(1, A[int(X[i - 1]), 1])

# weird that I can interchange A and B?
# now generate emissions
O = np.empty(T)
for i in range(T):
    # we add +2 so that emissions will be 2,3
    # so we don't confuse with underlying hidden chain
    O[i] = 2 + np.random.binomial(1, B[int(X[i]), 1])

posterior(A, np.array([.2, .8]), O)

array([[0.2       , 0.8       ],
       [0.26      , 0.74      ],
       [0.248     , 0.752     ],
       [0.2504    , 0.7496    ],
       [0.24992   , 0.75008   ],
       [0.250016  , 0.749984  ],
       [0.2499968 , 0.7500032 ],
       [0.25000064, 0.74999936],
       [0.24999987, 0.75000013],
       [0.25000003, 0.74999997]])